# Fun with Exoplanets
## Data Wrangling Part II: 

In [10]:
# import libraries
import pandas as pd
import numpy as np

from KOIclass import KOIObject

from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin


#### Load cumulative table and set KIC_ID and TCE_num will be set to a multi-index.

In [3]:
KOI_cumulative = pd.read_csv('..\data\KOI_cumulative.csv')
KOI_cumulative = KOI_cumulative.set_index(['KIC_ID', 'TCE_num']).sort_index()
KOI_cumulative.head()

,,Period,Depth,Duration,target_label
KIC_ID,TCE_num,,,,
757450,1,8.884923,16053.4,2.07004,1
1026032,1,8.460439,76412.9,4.73492,2
1293031,1,0.539366,145.0,5.43400,3
1432214,1,161.788327,87750.0,5.31000,2
1432789,1,9.931463,398.6,5.67800,1


#### Construct empty dataframe with dict keys from DV feature extraction, then loop over all entries in KOI_cumulative and append feature dict to empty dataframe.

#### Outputs light curve extracted features to csv

In [4]:
q = KOIObject(1026032, 1) # takes an example from the dataset for column name extraction.
cols = q.total_initialize().extract_allfeatures().keys()
# generate empty dataframe 
feat_df = pd.DataFrame(columns = cols)

for x in KOI_cumulative.index:
    KICID, TCE_num = x
    feat_df = feat_df.append(pd.Series(KOIObject(KICID, TCE_num).total_initialize().extract_allfeatures()).to_frame().T)

feat_df = feat_df.reset_index(drop = True).set_index(['KIC_ID', 'TCE_num'])
total_set = KOI_cumulative.join(feat_df)
feature_df_path = "..\\data\\feat_df_tot.csv"
total_set.to_csv(feature_df_path)

otebooks\KOIclass.py:344: PeakPropertyWarning: some peaks have a prominence of 0
  peak_loc_array, fit_dict = find_peaks(-noprimary, prominence = num_sigma*noprimary.std(), wlen = window_length)
c:\Users\prave\Springboard\Capstone2\notebooks\KOIclass.py:344: PeakPropertyWarning: some peaks have a prominence of 0
  peak_loc_array, fit_dict = find_peaks(-noprimary, prominence = num_sigma*noprimary.std(), wlen = window_length)
c:\Users\prave\Springboard\Capstone2\notebooks\KOIclass.py:344: PeakPropertyWarning: some peaks have a prominence of 0
  peak_loc_array, fit_dict = find_peaks(-noprimary, prominence = num_sigma*noprimary.std(), wlen = window_length)
c:\Users\prave\Springboard\Capstone2\notebooks\KOIclass.py:344: PeakPropertyWarning: some peaks have a prominence of 0
  peak_loc_array, fit_dict = find_peaks(-noprimary, prominence = num_sigma*noprimary.std(), wlen = window_length)
c:\Users\prave\Springboard\Capstone2\notebooks\KOIclass.py:344: PeakPropertyWarning: some peaks have a pro

#### So we have our dataframe with extracted features, 141 point phase-binned/normalized transit closeup series, and labels indexed by KIC ID and TCE number. 

In [5]:
total_set.head()

,,Period,Depth,Duration,target_label,even_odd_stat,p_secondary,time_complexity,rms,max,min,...,LCBIN_131,LCBIN_132,LCBIN_133,LCBIN_134,LCBIN_135,LCBIN_136,LCBIN_137,LCBIN_138,LCBIN_139,LCBIN_140
KIC_ID,TCE_num,,,,,,,,,,,,,,,,,,,,,
757450,1,8.884923,16053.4,2.07004,1,0.925362,0.007862,12.181757,0.001264,0.000177,-0.015739,...,0.003811,0.003485,0.004310,0.005135,0.005959,0.006784,0.007609,0.008434,0.009258,0.010083
1026032,1,8.460439,76412.9,4.73492,2,0.978880,0.000000,6.820030,0.008070,0.000360,-0.077604,...,0.001283,0.001274,0.001265,0.001134,0.001003,0.000872,0.000741,0.000870,0.000999,0.001129
1293031,1,0.539366,145.0,5.43400,3,0.499758,0.289257,8.781153,0.000008,0.000026,-0.000017,...,0.417232,0.435126,0.453019,0.470913,0.488807,0.506700,0.524594,0.542487,0.560381,0.578274
1432214,1,161.788327,87750.0,5.31000,2,0.997538,0.000004,29.683030,0.002478,0.001184,-0.086902,...,-0.000726,-0.000781,-0.000837,-0.000892,-0.000278,0.000336,0.000950,0.001251,0.001553,0.001854
1432789,1,9.931463,398.6,5.67800,1,0.299867,0.008181,20.743805,0.000072,0.000360,-0.000422,...,0.182302,0.196967,0.202510,0.208052,0.213595,0.153873,0.094150,0.034427,0.109613,0.184799


In [7]:
total_set.shape

(5355, 151)

#### Let's check for NaNs.

In [9]:
total_set.isna().sum().sort_values(ascending = False)

even_odd_stat    16
LCBIN_140         0
LCBIN_36          0
LCBIN_43          0
LCBIN_42          0
                 ..
LCBIN_89          0
LCBIN_88          0
LCBIN_87          0
LCBIN_86          0
Period            0
Length: 151, dtype: int64

There are some NaNs in the even-odd phase transit statistic. This is due to the fact that there are some light curves where there is only odd or even phase cycles (mainly TCEs with long periods and gaps in the time series such that one of the parities is completely absent). Let's just 
impute this column to 0.5 (i.e. no difference between even-odd parity).

In [18]:
total_set['even_odd_stat'] = total_set['even_odd_stat'].fillna(0.5)
total_set.isna().sum().sort_values(ascending = False)

LCBIN_140    0
LCBIN_45     0
LCBIN_43     0
LCBIN_42     0
LCBIN_41     0
            ..
LCBIN_89     0
LCBIN_88     0
LCBIN_87     0
LCBIN_86     0
Period       0
Length: 151, dtype: int64

#### Overwrite csv with no nans: can use this to start our EDA

In [19]:
feature_df_path = "..\\data\\feat_df_tot.csv"
total_set.to_csv(feature_df_path)

In [31]:
S = KOIObject(1026032, 1).total_initialize()